In [16]:
import pandas as pd

df_br = pd.read_csv('Dados/campeonato-brasileiro-full.csv')

df_br['data'] = pd.to_datetime(df_br['data'], dayfirst=True)  

jogos_2017 = df_br[df_br['data'].dt.year == 2017]

jogos_sao_paulo_2017 = jogos_2017[(jogos_2017['mandante'] == 'Sao Paulo') | (jogos_2017['visitante'] == 'Sao Paulo')]

def determinar_vencedor(win):
    if win['mandante_Placar'] > win['visitante_Placar']:
        return win['mandante']
    elif win['mandante_Placar'] < win['visitante_Placar']:
        return win['visitante']
    else:
        return 'Empate'

jogos_sao_paulo_2017['resultado'] = jogos_sao_paulo_2017.apply(determinar_vencedor, axis=1)

colunas_filtros = ['rodata', 'mandante', 'visitante', 'mandante_Placar', 'visitante_Placar', 'resultado']

print(jogos_sao_paulo_2017[colunas_filtros].to_string(index=False))


 rodata     mandante    visitante  mandante_Placar  visitante_Placar    resultado
      1     Cruzeiro    Sao Paulo                1                 0     Cruzeiro
      2    Sao Paulo         Avai                2                 0    Sao Paulo
      3    Sao Paulo    Palmeiras                2                 0    Sao Paulo
      4  Ponte Preta    Sao Paulo                1                 0  Ponte Preta
      5    Sao Paulo      Vitoria                2                 0    Sao Paulo
      6  Corinthians    Sao Paulo                3                 2  Corinthians
      7        Sport    Sao Paulo                0                 0       Empate
      8    Sao Paulo  Atletico-MG                1                 2  Atletico-MG
      9 Athletico-PR    Sao Paulo                1                 0 Athletico-PR
     10    Sao Paulo   Fluminense                1                 1       Empate
     11     Flamengo    Sao Paulo                2                 0     Flamengo
     12       Sa

C:\Users\ugoma\AppData\Local\Temp\ipykernel_16792\752986191.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jogos_sao_paulo_2017['resultado'] = jogos_sao_paulo_2017.apply(determinar_vencedor, axis=1)
